In [5]:
import importlib
import cv2
import json
import os
import sys

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')
sys.path.append(NOMEROFF_NET_DIR)

def compare(model_path="../../../models/anpr_ocr_ru_6-gpu.h5", 
           text_detector_name = "ru",
           img_format = "png",
           root_dir='/mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr', # ../../../datasets/ocr/kz/kz2
           predicted_part_size=1000,
           replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}):
    text_detector_module = importlib.import_module("NomeroffNet.TextDetectors."+text_detector_name)
    text_detector = getattr(text_detector_module, text_detector_name)()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dirName, subdirList, fileList in os.walk(ann_dir):
        for fname in fileList:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []                
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        predicted_part = text_detector.predict(part_imgs)
        predicted += predicted_part

    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        jsonData["moderation"]["predicted"] = predicted_item
        if jsonData["description"] == jsonData["moderation"]["predicted"]:
            #jsonData.update(replace_tamplate)
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}', real: '{}' in file {}".format(jsonData["moderation"]["predicted"],jsonData["description"], json_path))
            err_cnt = err_cnt+1

        jsonData["description"] = predicted_item
            
        with open(json_path, "w") as jsonWF:
            json.dump(jsonData, jsonWF)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [6]:
compare()

LOADED 1878 ANNOTATIONS
LOADED 1878 IMAGES
PREDICTED 1878 IMAGES
Predicted 'A337BYE6', real: 'Dnp9NlAgohK5tHisrdfezLj640vVya7QEuFYZCbSOGqJ1T8X3I' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/Dnp9NlAgohK5tHisrdfezLj640vVya7QEuFYZCbSOGqJ1T8X3I.json
Predicted 'A717AA33', real: 'HTobVBx3AyFsCPuzgfqXdr89KnO5L0YmpjUZvIMWwJklERa6c7' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/HTobVBx3AyFsCPuzgfqXdr89KnO5L0YmpjUZvIMWwJklERa6c7.json
Predicted 'XO1249', real: 'idJu5A92Un0xvXLptEDOafQTjYFcwIyGrCmboM6NeRq7KVls8W' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/idJu5A92Un0xvXLptEDOafQTjYFcwIyGrCmboM6NeRq7KVls8W.json
Predicted '7315T5', real: 'EYh3joBmlM5ReDVJbinIFqZXGvL78gQKA2t1adkrs0WpHc9xwf' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/EYh3joBmlM5ReDVJbinIFqZXGvL78gQKA2t1adkrs0WpHc9xwf.json
Predicted 'B408CM37', real: 'FmMzrqGfCl

Predicted 'TP9863', real: '4otQr2j58XsKBbOkWRCMeizy6vFnPTS7Uw9amL3JNpI1dxVZf0' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/4otQr2j58XsKBbOkWRCMeizy6vFnPTS7Uw9amL3JNpI1dxVZf0.json
Predicted 'C27H6', real: 'FNb2RgMuyP14SLwJ75zGjsT3D8ZCWmAeaBY9cqfIhHtilXQdoE' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/FNb2RgMuyP14SLwJ75zGjsT3D8ZCWmAeaBY9cqfIhHtilXQdoE.json
Predicted 'A677B5', real: 'QIOz1RrGPeMjqTJhsYcup3l9FnLNv5f6HtKxASmyEaWDZdiU04' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/QIOz1RrGPeMjqTJhsYcup3l9FnLNv5f6HtKxASmyEaWDZdiU04.json
Predicted 'A8P8A5', real: 'L6W82iGsPzv71TboXNwKOqhBdZVMI5c3pJayfRHSD9tmjAeEY0' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/L6W82iGsPzv71TboXNwKOqhBdZVMI5c3pJayfRHSD9tmjAeEY0.json
Predicted '3', real: 'Qfy1YgNvPwe8CLZ29sxuIUpTErSA6jchKXl7DHdOa4Rzkt3JV0' in file /mnt/data/home/nn/datasets/

Predicted 'B621TH3', real: 'IL9z4YTMtsqedf3NicX08ykHCOuoJWFG7hZ5xBP1ljS26ADEUv' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/IL9z4YTMtsqedf3NicX08ykHCOuoJWFG7hZ5xBP1ljS26ADEUv.json
Predicted 'T9BX9', real: 'QvqiOnmkhYMp4ZPswC3EgeuLySIDHRdA56c7fz2NxFj0TWBoXJ' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/QvqiOnmkhYMp4ZPswC3EgeuLySIDHRdA56c7fz2NxFj0TWBoXJ.json
Predicted 'E0399H80', real: 'saz7HPKk8QMvNtx5bnT3Olmu4qo0hJ1CAYFeSdfRUyGj9IVB6r' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/saz7HPKk8QMvNtx5bnT3Olmu4qo0hJ1CAYFeSdfRUyGj9IVB6r.json
Predicted 'X4441TE', real: 'ClbB51NsixfW27ya30TwForKdEAXDv4cPZJ6GqMeL8YtuSpVRm' in file /mnt/data/home/nn/datasets/autoriaNumberplateOptions3Dataset-2019-10-04/lnr/ann/ClbB51NsixfW27ya30TwForKdEAXDv4cPZJ6GqMeL8YtuSpVRm.json
Predicted 'A7222TE', real: 'mArY9IylkFKJN5TwhWQtzuoLMVD40Od68qegxaSfcRbPsHEBG2' in file /mnt/data/home/nn